<a href="https://colab.research.google.com/github/InstitutodaVinci3/Vancouver/blob/master/Producao_Diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importando as bibliotecas básicas e preparando ambiente
import pandas as pd                     # convenção 
import numpy as np                      # convenção 
import random as rd                     # rd não é alias padrão mas sim adotado aqui somente 
# Itens adicionais
import glob
import os
import re

# Teste 1 - com os TXT

In [ ]:
# ***** Lembrar que é necessário "MONTAR" o seu Google Dive junto ao Jupiter Collab como pré-requisito ****
path =r'/content/drive/MyDrive/Colab Notebooks/MedMep DataSet/Prod Diaria txt'
filenames = glob.glob(path + "/*.txt")

In [ ]:
# junção dos arquivos de Produção Diária - arquivos TXT
dfs =[]
for filename in filenames:
  a = pd.read_table(filename, sep='\t', encoding='latin1', engine='python', index_col=0)
  dfs.append(a[:-1])

# Concatenando todos os dados em um único DataFrame
Producao_df = pd.concat(dfs, ignore_index=True)
Producao_df
Producao_df.to_excel("Producao_Diaria.xlsx")

# Teste 2 - com os XLSX

In [12]:
# Testando com os arquivos XLSX
path =r'/content/drive/MyDrive/Prod Diaria XLS'
filenames = glob.glob(path + "/*.xlsx")

In [4]:
a = pd.read_excel('/content/drive/MyDrive/Prod Diaria XLS/Produção diária_20170503T172000.xlsx', sheet_name=0,header=0)
a.head()

,Group Number,Animal Number,Lactation Number,Days In Milk,Peak Yield,Milk Time Today For Session 1,Duration Today Session 1 (mm:ss),Yield Today Session 1,Milk Time Today For Session 2,Duration Today Session 2 (mm:ss),Yield Today Session 2,Start Time Session 1 Yesterday,Duration Yesterday Session 1 (mm:ss),Yield Yesterday Session 1,Start Time Session 2 Yesterday,Duration Yesterday Session 2 (mm:ss),Yield Yesterday Session 2
0,NaN,2646,1,370,11.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,5409,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,5402,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,5403,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,5407,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Para extrair da Data do nome do arquivo
dia = re.search(r'_(\d+)', '/content/drive/MyDrive/Prod Diaria XLS/Produção diária_20170503T172000.xlsx')
print(dia[1])

20170503


In [16]:
# junção dos arquivos de Produção Diária - arquivos XLSX
dfs = []
for filename in filenames:
#  dia = re.search(r'_(\d+)', filename)
  a = pd.read_excel(filename, sheet_name=0, header=0)
#  a.head()
  dfs.append(a)
Producao_df = pd.concat(dfs) # Concatenando todos os dados em um único DataFrame
Producao_df.to_excel("/content/drive/MyDrive/Prod Diaria XLS/Producao_Diaria.xlsx")

#Producao_df = pd.concat(dfs, ignore_index=True)
#  dfs
#  dfs.append(a[:-1])
#Producao_df.to_excel("Producao_Diaria.xlsx")